In [2]:
import pandas as pd


In [1]:
import detect_ai_content
import os 
import pickle

module_dir_path = os.path.dirname(detect_ai_content.__file__)
model_path =f'{module_dir_path}/../detect_ai_content/models/linchenpal/best_xgb_model.pkl'
with open(model_path, 'rb') as file:
    model = pickle.load(file)

model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [31]:
df = pd.read_csv("test_data/test_output_internalfeature.csv")
df
                 

,generated,stopwords_ratio,punctuation_ratio,repetition_ratio,dependency_ratio,spelling_errors_ratio,pos__ratio,avg_word_length,lexical_diversity,flesch_reading_ease,smog_index,flesch_kincaid_grade,sentiment
0,1.0,0.331818,0.086364,0.072727,0.875000,0.081818,1.009091,5.050000,0.540909,37.84,16.1,14.1,-1
1,0.0,0.518000,0.030000,0.150000,0.272727,0.020000,1.000000,4.212000,0.274000,52.91,11.6,16.6,-1
2,0.0,0.531818,0.061364,0.125000,0.050000,0.072727,1.002273,3.740909,0.350000,77.87,8.0,7.0,1
3,1.0,0.422961,0.081571,0.090634,0.230769,0.078550,1.018127,4.728097,0.483384,47.93,13.1,12.3,1
4,0.0,0.356564,0.102107,0.047002,0.964286,0.118314,0.998379,4.162075,0.445705,69.62,11.1,8.1,-1


In [33]:
df.columns

Index(['generated', 'stopwords_ratio', 'punctuation_ratio', 'repetition_ratio',
       'dependency_ratio', 'spelling_errors_ratio', 'pos__ratio',
       'avg_word_length', 'lexical_diversity', 'flesch_reading_ease',
       'smog_index', 'flesch_kincaid_grade', 'sentiment'],
      dtype='object')

In [32]:
df.shape

(5, 13)

In [20]:
df.columns

Index(['generated', 'word_count_ratio', 'stopwords_ratio', 'punctuation_ratio',
       'repetition_ratio', 'bigram_count_ratio', 'trigram_count_ratio',
       'dependency_ratio', 'spelling_errors_ratio', 'pos__ratio',
       'pos_SPACE_ratio', 'avg_word_length', 'lexical_diversity',
       'flesch_reading_ease', 'smog_index', 'flesch_kincaid_grade', 'polarity',
       'subjectivity', 'sentiment'],
      dtype='object')

In [21]:
X = df.iloc[:, 1:]
y = df['generated']

In [22]:
# Set correlation threshold
correlation_threshold = 0.1

# Step 1: Calculate correlation with the 'generated' column and filter features
correlation_with_target = df.corr()['generated'].abs()
selected_features = correlation_with_target[correlation_with_target > correlation_threshold].index.drop('generated')

# Filter the dataset to only include selected features
X = df[selected_features]
y = df['generated']

In [27]:
# there is error
# training data did not have the following fields: word_count_ratio, bigram_count_ratio, polarity, pos_SPACE_ratio, trigram_count_ratio, subjectivity 
# removed some columns according to error 

X = X.drop(columns=["word_count_ratio", "bigram_count_ratio", "polarity", "pos_SPACE_ratio", "trigram_count_ratio", "subjectivity" ])
X.shape


(5, 12)

In [28]:
X

,stopwords_ratio,punctuation_ratio,repetition_ratio,dependency_ratio,spelling_errors_ratio,pos__ratio,avg_word_length,lexical_diversity,flesch_reading_ease,smog_index,flesch_kincaid_grade,sentiment
0,0.331818,0.086364,0.072727,0.875000,0.081818,1.009091,5.050000,0.540909,37.84,16.1,14.1,-1
1,0.518000,0.030000,0.150000,0.272727,0.020000,1.000000,4.212000,0.274000,52.91,11.6,16.6,-1
2,0.531818,0.061364,0.125000,0.050000,0.072727,1.002273,3.740909,0.350000,77.87,8.0,7.0,1
3,0.422961,0.081571,0.090634,0.230769,0.078550,1.018127,4.728097,0.483384,47.93,13.1,12.3,1
4,0.356564,0.102107,0.047002,0.964286,0.118314,0.998379,4.162075,0.445705,69.62,11.1,8.1,-1


In [30]:
model.predict(X)

array([1, 0, 0, 1, 0])